# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [110]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [111]:
csv_weather_path = "../WeatherPy/weather_search.csv"
weather_df = pd.read_csv(csv_weather_path)

In [112]:
weather_df

,city,city_id,country,latitude,longitude,temperature,max_temp,cloudiness,humidity,wind_speed
0,São Gabriel da Cachoeira,3662342,BR,-0.1303,-67.0892,73.67,73.67,95,97,0.78
1,Portland,5746545,US,45.5234,-122.6762,49.84,51.01,100,84,3.00
2,Sovetskiy,1491230,RU,61.3614,63.5842,44.15,44.15,100,66,9.10
3,Albany,5106841,US,42.6001,-73.9662,50.31,53.01,100,45,1.83
4,Fairbanks,5861897,US,64.8378,-147.7164,61.20,62.60,1,23,3.44
...,...,...,...,...,...,...,...,...,...,...
556,Uvarovo,477626,RU,51.9849,42.2615,36.00,36.00,29,77,16.53
557,Chingirlau,610091,KZ,51.0950,54.0814,48.97,48.97,77,77,17.00
558,Avallon,3035768,FR,47.4833,3.9000,46.00,46.00,72,67,6.55
559,Medininagar,1273626,IN,24.0333,84.0667,97.92,97.92,1,9,6.26


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [121]:
gmaps.configure(api_key=g_key)

locations = weather_df[["latitude","longitude"]].astype(float)
humidity=weather_df["humidity"].astype(float)

figure_layout = {
    'width': '500px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,map_type='ROADMAP')

heat_layer = gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=100,point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [125]:
vacay_options=weather_df[(weather_df['max_temp']>70)&(weather_df['max_temp']<80)&(weather_df['humidity']<30)&(weather_df['cloudiness']<30)]
vacay_options

,city,city_id,country,latitude,longitude,temperature,max_temp,cloudiness,humidity,wind_speed
48,Cabo San Lucas,3985710,MX,22.8909,-109.9124,71.53,72.00,1,21,10.22
70,Saint George,5546220,US,37.1041,-113.5841,74.08,75.20,1,15,14.97
140,Adrar,2381972,MR,20.5022,-10.0711,75.15,75.15,6,16,8.48
317,Alice Springs,2077895,AU,-23.7000,133.8833,73.72,73.99,0,25,10.36
391,Yulara,6355222,AU,-25.2406,130.9889,73.40,73.40,0,27,16.11
462,Mīrābād,1133310,AF,30.4362,61.8383,79.29,79.29,15,14,18.84
469,Birin,1283334,NP,26.8865,88.0718,77.05,77.05,2,24,5.93
515,Khāsh,1160571,IR,28.2211,61.2158,75.99,75.99,0,14,0.83


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [176]:
hotel_df=pd.DataFrame(vacay_options,columns=["city","country"])
hotel_df["hotel_name"]=""
hotel_df['hotel_lat']=""
hotel_df["hotel_lon"]=""
hotel_df.head()

,city,country,hotel_name,hotel_lat,hotel_lon
48,Cabo San Lucas,MX,,,
70,Saint George,US,,,
140,Adrar,MR,,,
317,Alice Springs,AU,,,
391,Yulara,AU,,,


In [177]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
location="22.8909,-109.9124"
target_radius = 8000
target_type = "hotel"
key= g_key

target_url = f"{base_url}&location={location}&radius={target_radius}&type={target_type}&key={g_key}"
response = requests.get(target_url).json()
print(json.dumps(response,indent=4,sort_keys=True))


{
    "html_attributions": [],
    "next_page_token": "ATtYBwIYIiO60VKYTDtJ9Yw9O8ihii5uYFIsBbclfalDvFVeuBYisDZ0Va94V6bSPthcMhbdPnXmSUgsdIC6qB88mFawYI5OeMjoClhAl-qzXuEiP-O5edoMlFdZAf5OW7Sh-hevepBaRIf4rAjTKVObcANsFvIO-vIoK3msfXTziTZ4DMwbB26-6AY66JcTDsO-ExxHdDu0BL7x5Ps1QRBTokKShyftngZWfzxzeJ1zBkxDJQZx3h6ug1r2L332q5k64TFDj9vK83SPIbd6mTgrUmu3F0jILNztrRdcGgtQrHhZyvUYkJpJ8a_UshP-NwRidsE2monPKy3ky-BGT6EI6gXyrdD7KeTWHeAJ8yowiTOLNlLrrA4V65-3AaUvfC9IeTvJGGf_0Or9C_7St-5C68oxBjpCJActFucj82bZgBG8TtAk04CudA",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 22.8905327,
                    "lng": -109.9167371
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.92028757787479,
                        "lng": -109.8942558770843
                    },
                    "southwest": {
                        "lat": 22.87216499597992,
                        "lng": -109.9717402828

In [161]:
response['results'][1]['name']

'Hotel Tesoro Los Cabos'

In [178]:
hotel_name = []

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {
    "radius": 8000,
    "type": "hotel",
    "key": g_key
}

for index, row in vacay_options.iterrows():
    #get latitude & longitude
    lat=row['latitude']
    lon=row['longitude']
    
    params["location"]=f"{lat},{lon}"
            
    response = requests.get(base_url, params=params).json()
    
   
    try:
        print(f"For {city} the closest hotel is {response['results'][1]['name']}.")
            
        hotel_df.loc[index, 'hotel_name'] = response['results'][1]['name']
        hotel_df.loc[index, 'hotel_lat'] = response['results'][1]['geometry']['location']['lat']
        hotel_df.loc[index, 'hotel_lon'] = response['results'][1]['geometry']['location']['lng']
    
    except IndexError:
        print(f"No hotel avaialable in {city}")
    
    print("---------------------")


For 559 the closest hotel is Hotel Tesoro Los Cabos.
---------------------
For 559 the closest hotel is Best Western Plus Abbey Inn.
---------------------
No hotel avaialable in 559
---------------------
For 559 the closest hotel is Desert Palms Alice Springs.
---------------------
For 559 the closest hotel is Desert Gardens Hotel - Ayers Rock Resort.
---------------------
For 559 the closest hotel is Deh-e Kadila.
---------------------
For 559 the closest hotel is Rastriya Banijya Bank.
---------------------
For 559 the closest hotel is Badran Gostaran Co..
---------------------


In [181]:
hotel_df


,city,country,hotel_name,hotel_lat,hotel_lon
48,Cabo San Lucas,MX,Hotel Tesoro Los Cabos,22.8809,-109.912
70,Saint George,US,Best Western Plus Abbey Inn,37.089,-113.584
140,Adrar,MR,,,
317,Alice Springs,AU,Desert Palms Alice Springs,-23.7134,133.88
391,Yulara,AU,Desert Gardens Hotel - Ayers Rock Resort,-25.2437,130.985
462,Mīrābād,AF,Deh-e Kadila,30.4753,61.797
469,Birin,NP,Rastriya Banijya Bank,26.8952,88.0691
515,Khāsh,IR,Badran Gostaran Co.,28.2189,61.213


In [185]:
hotel_df.columns = ['City', 'Country', 'Hotel Name', 'Lat','Lng']

In [186]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
